# Data Preparation and Imputation

<b> As we have completed the Exploratory Data Analysis, we have obtained an overview on the specifics of each attributes in the dataset.
Further on, we proceed to handle outliers and missing values in few attributes that were found from previous EDA.</b>

In [1]:
# import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import random
import warnings

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [2]:
path = 'C:/Users/MANEET/bank_marketing'
data_add = '/data'
report_add = '/report'
figures_add = '/figures'
experiment_add = '/experiments'

In [3]:
df = pd.read_csv(path + data_add +'./raw/bank-additional-full.csv', delimiter = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


<b>Outliers </b> : In statistics outleirs or edge cases are defined values excludeing the range of 1.5 * Q3 of the sample. Based on the EDA from the previous steps "Age" and "Campaign" have outliers. But the both the variables are actually following the real world siutation and hence we cannot consider them as outliers.

## Data Imputation

From the Occupation versus Age plot, it is evident that there are a few unknown and unemployed samples for higher age groups. Considering the data generated from Portugese Banking Institution, we assume that most clients belong to Portugese nationaltiy. The legal retirement age is 65 for employees in portugese and so we replace the age groups above 65 to retired.

In [5]:
df['job'][df['age']>65].value_counts()

retired          533
housemaid         32
admin.            14
unknown           10
management         9
technician         6
blue-collar        5
entrepreneur       4
unemployed         3
self-employed      2
services           1
Name: job, dtype: int64

In [6]:
df.loc[(df["age"]>65) & (df["job"]=='unknown'), 'job'] = 'retired'
df.loc[(df["age"]>65) & (df["job"]=='unemployed'), 'job'] = 'retired'

As we know the dataset is obtained from real world instances, we can generate relation between education and occupation. As there are high "unknown" instances in job and education, we can hypothesize education based on job and vice-versa.

In [7]:
job_education_ct = pd.crosstab(df.job,df.education)

In [8]:
job_education_ct

education,basic.4y,basic.6y,basic.9y,high.school,illiterate,professional.course,university.degree,unknown
job,,,,,,,,
admin.,77,151,499,3329,1,363,5753,249
blue-collar,2318,1426,3623,878,8,453,94,454
entrepreneur,137,71,210,234,2,135,610,57
housemaid,474,77,94,174,1,59,139,42
management,100,85,166,298,0,89,2063,123
retired,601,75,145,276,3,241,286,106
self-employed,93,25,220,118,3,168,765,29
services,132,226,388,2682,0,218,173,150
student,26,13,99,357,0,43,170,167


In [9]:
job_education_max = job_education_ct.idxmax(axis=1) #this stores key-value pair of job-education to predict education
education_job_max = job_education_ct.idxmax(axis=0)#this stores key-value pair of educaiton-job to predict job

In [10]:
education_job_max.pop('unknown')
job_education_max.pop('unknown')

'unknown'

In [11]:
# Replacing unknowns in "job" and "education"

In [12]:
df.loc[(df['education']=='unknown') & (df['job']=='unknown'),'job'] = random.choice(education_job_max)

In [13]:
for i in education_job_max.keys():
    df.loc[(df['education'] =='unknown') & (df['job'] == i),'education'] = education_job_max[i]

In [14]:
for i in job_education_max.keys():
    df.loc[(df['job'] =='unknown') & (df['education'] == i),'job'] = job_education_max[i]

<b> Inferences </b>
After this operation, following replacements were conducted in the Education and Job fields.

Occupation based on Education.
- Management> High School
- Service > High School
- House Maid > Basic

Education based on Occupation
- Basic 4y, 6y, 9y > blue collar
- Professional_course > technician

Pdays attribute has < 5% of the positive values and so we will drop them as an edge case.

In [15]:
df_copy = df.drop('pdays',axis=1)
df_copy.head() 
df = df_copy

In [16]:
df.to_csv(path + data_add + '/processed/data_imputation.csv')

## Data Preprocessing <a class="anchor" id="six-bullet"></a>

In [17]:
train_df = df.loc[:,df.columns !='y']
target_df = df.loc[:,df.columns == 'y']
train_df.to_csv(path + data_add + '/processed/Raw_sample_features.csv')
target_df.to_csv(path + data_add + '/processed/Raw_sample_labels.csv')

In [18]:
# writing helper function differentiate Categorical and Numerical columns
def find_cat_cols(df):
    num_cols = list(df._get_numeric_data().columns)
    cat_cols = list(set(df.columns) - set(num_cols))
    print("Found {0} Numerical columns in DataFrame".format(len(num_cols)))
    print("Found {0} Categorical columns in DataFrame".format(len(cat_cols)))
    return num_cols, cat_cols
    

In [19]:
numeric_features, categorical_features = find_cat_cols(train_df)

Found 9 Numerical columns in DataFrame
Found 10 Categorical columns in DataFrame


In [20]:
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

In [21]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_over,y_over = oversample.fit_resample(train_df,target_df)
X_over.to_csv(path + data_add + '/processed/Over_sample_features.csv')
y_over.to_csv(path + data_add + '/processed/Over_sample_labels.csv')

In [22]:
X_over.shape, y_over.shape

((73096, 19), (73096, 1))

In [23]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_under,y_under = undersample.fit_resample(train_df,target_df)
X_under.to_csv(path + data_add + '/processed/Under_sample_features.csv')
y_under.to_csv(path + data_add + '/processed/Under_sample_labels.csv')

In [24]:
print('Oversampled dataset shape %s' % len(X_over))
print('Undersampled dataset shape %s' % len(X_under))

Oversampled dataset shape 73096
Undersampled dataset shape 9280


In [25]:
def preprocess_data(X,y):
    numeric_features, categorical_features = find_cat_cols(X)
    scaler = MinMaxScaler()
    label_encoder = LabelEncoder()
    X[numeric_features] = scaler.fit_transform(X[numeric_features])
    y_preprocessed = pd.Series(label_encoder.fit_transform(y))
    X_preprocessed = pd.concat([X,pd.get_dummies(X[categorical_features])],axis=1)
    X_preprocessed.drop(labels = categorical_features,axis=1,inplace=True)
    return X_preprocessed, y_preprocessed

In [26]:
X_over_preprocessed, y_over_preprocessed = preprocess_data(X_over,y_over)
print(X_over_preprocessed.shape, y_over_preprocessed.shape)
X_over_preprocessed.to_csv(path + data_add + '/processed/Over_processed_features.csv')
y_over_preprocessed.to_csv(path + data_add + '/processed/Over_processed_labels.csv')

Found 9 Numerical columns in DataFrame
Found 10 Categorical columns in DataFrame
(73096, 62) (73096,)


In [27]:
X_under_preprocessed,y_under_preprocessed = preprocess_data(X_under,y_under)
print(X_under_preprocessed.shape, y_under_preprocessed.shape)
X_under_preprocessed.to_csv(path + data_add + '/processed/Under_processed_features.csv')
y_under_preprocessed.to_csv(path + data_add + '/processed/Under_processed_labels.csv')

Found 9 Numerical columns in DataFrame
Found 10 Categorical columns in DataFrame
(9280, 61) (9280,)


In [28]:
X_raw_preprocessed,y_raw_preprocessed = preprocess_data(train_df,target_df)
print(X_raw_preprocessed.shape,y_raw_preprocessed.shape)
X_raw_preprocessed.to_csv(path + data_add + '/processed/Raw_processed_features.csv')
y_raw_preprocessed.to_csv(path + data_add + '/processed/Raw_processed_labels.csv')

Found 9 Numerical columns in DataFrame
Found 10 Categorical columns in DataFrame
(41188, 62) (41188,)


### Lets head to Modelling and Evaluation phase which is continued in other notebook